In [70]:
def is_prime(n):
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

In [71]:
def set_if_inside (m, x,y,value):
    rows, cols = m.shape
    if 0 <= x < rows and  0 <= y < cols:
        m[x, y] = value
        return True
    return False

In [72]:
def fill_primes(m,offset):
    rows, cols = m.shape
    mid = round(rows * 0.5)
    incr = 1
    side = 0
    i = offset
    x = mid
    y = mid
    set_if_inside (m,x,y,2)
    while i < rows * cols:
        side += 1
        for j in range (side):
            x += incr
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        for j in range (side):
            y += incr
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        side += 1
        for j in range (side):
            x -= incr
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
        for j in range (side):
            y -= incr
            if is_prime(i):
                set_if_inside(m, x, y, 1)
            i += 1
    return m

In [73]:
def display_primes(matrix, size,offset):
    # matrix = np.zeros((size+10,size+10))
    matrix = fill_primes(matrix,offset)
    ys, xs = np.where(matrix == 1)
    y0, x0 = np.where(matrix == 2)
    
    with output:
        clear_output(wait=True)
        plt.close('all')  
        plt.figure(figsize=(15/1000*size,15/1000*size))
        plt.scatter(x0, y0, c='red', s=1)
        plt.scatter(xs, ys, c='blue', s=1)
        plt.xlim(0,size)
        plt.ylim(0,size)
        plt.grid(False)
        plt.show()
    return matrix

In [74]:
def fourier_lowpass(matrix, radius):
    # 1. Fourier
    f = np.fft.fft2(matrix)
    fshift = np.fft.fftshift(f)

    rows, cols = matrix.shape
    crow, ccol = rows//2, cols//2

    # 2. Low-pass maszk (kör alakú)
    Y, X = np.ogrid[:rows, :cols]
    dist = np.sqrt((X - ccol)**2 + (Y - crow)**2)
    mask = dist <= radius

    # 3. Szűrés
    fshift_filtered = fshift * mask

    # 4. Inverz FFT
    img_back = np.fft.ifft2(np.fft.ifftshift(fshift_filtered))
    return np.abs(img_back)


In [75]:
def fourier_highpass(matrix, radius):
    f = np.fft.fft2(matrix)
    fshift = np.fft.fftshift(f)

    rows, cols = matrix.shape
    crow, ccol = rows//2, cols//2

    Y, X = np.ogrid[:rows, :cols]
    dist = np.sqrt((X - ccol)**2 + (Y - crow)**2)
    mask = dist > radius   # csak a magas frekvenciákat tartjuk meg

    fshift_filtered = fshift * mask

    img_back = np.fft.ifft2(np.fft.ifftshift(fshift_filtered))
    return np.abs(img_back)

In [76]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

initial_limit = 100000
initial_offset = 1

output = widgets.Output()
matrix = None


# --- Data and initial plot ---
size = round(initial_limit ** 0.5)
matrix = np.zeros((size+10,size+10))

# --- Widgets ---
limit_text = widgets.IntText(
    value=initial_limit,
    description='Checks primes up to:',
    placeholder='Enter a number',
)

offset_text = widgets.IntText(
    value=initial_offset,
    description='Offset:',
    placeholder='Enter a number',
)


button = widgets.Button(
    description="Display primes",
    button_style="info",  # optional (success, info, warning, danger)
    tooltip='Use the number in the text box',
)

# --- Button for FFT ---
button_fft = widgets.Button(
    description="FFT",
    button_style="info",  # optional (success, info, warning, danger)
    tooltip='Use the number in the text box',
)

# --- Callback: update the existing ---
def on_button_clicked(button):
    global matrix
    try:
        size = round(limit_text.value**0.5)
        matrix = np.zeros((size+10,size+10))
        matrix = display_primes(matrix, size,offset_text.value)
    except ValueError:
        size = round (initial_limit ** 0.5)
        print("Invalid input, using size " )

def on_button_fft_clicked(button_fft):
    global matrix
    try:
        # --- Teszt hívás: matrix-ra futtatjuk ---
        # low-pass
        lp1 = fourier_lowpass(matrix, radius=20)
        lp2 = fourier_lowpass(matrix, radius=40)
        lp3 = fourier_lowpass(matrix, radius=60)
        # high-pass
        #hp = fourier_highpass(matrix, radius=100)
        # --- Megjelenítés ---
        with output:
            clear_output(wait=True)
            plt.close('all')
            plt.figure(figsize=(24,12))
            plt.subplot(1,3,1)
            plt.title("Low-pass (lágyított) radius = 20")
            plt.imshow(lp1, cmap='gray')
            plt.axis("off")
            plt.subplot(1,3,2)
            plt.title("Low-pass (lágyított) radius = 40")
            plt.imshow(lp2, cmap='gray')
            plt.axis("off")

            plt.subplot(1,3,3)
            plt.title("Low-pass (lágyított) radius = 60")
            plt.imshow(lp3, cmap='gray')
            plt.axis("off")
            plt.show()
    except ValueError:
        print("Error " )

# Bind the function to the button's click event
button.on_click(on_button_clicked)
button_fft.on_click(on_button_fft_clicked)


# --- Show UI ---
row1 = widgets.HBox([limit_text, offset_text])
row2 = widgets.HBox([button, button_fft])
display(widgets.VBox([row1, row2, output]))

# Display the map with a changing starting point - as an animation
#for j in range(100):
#    display_primes(size,j*2+1)
